# MongoDB

## Conexión con MongoDB y borrado de datos

In [1]:
import pymongo 
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError
import json

from pprintpp import pprint as pp

In [2]:
# connect to database
connection = MongoClient('localhost', 27017)

In [3]:
# Borramos la base de datos
connection.drop_database('black')

# Pymongo permite una sintaxis practicamente igual que la original
db = connection.black

## Lectura en Pandas

Leemos los datos desde el fichero Excel y los desnormalizamos

In [4]:
import pandas as pd
import datetime

In [5]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6823 non-null   object        
 6   actividad_completa  7623 non-null   object        
 7   actividad           7623 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6003 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


## Carga en MongoDB

Gracias a Pandas convertimos el dataset en formato JSON, que es el que vamos a insertar en MongoDB prácticamente sin cambios.

In [6]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [7]:
pp(json_list[0])

{
    'actividad': 'COCHE',
    'actividad_completa': 'GARAJES Y APARCAMIENTOS',
    'comercio': 'PARKING EGUISA C/SEVILLA',
    'fecha': 1174435200000,
    'funcion': 'concejal',
    'hora': 15,
    'id_miembro': 6,
    'importe': 3.2,
    'minuto': 26,
    'nombre': 'Antonio Romero Lázaro',
    'organizacion': 'PSOE',
}


En MongoDB vamos a crear una única colección donde vamos a guardar todos los datos desnormalizados:

<img src="images/Modelo%20MongoDB.png" width=300 height=100>

<br><br> 

En la inserción de la información en mongo realizan algunas tareas de limpieza:

- Eliminar los campos nulos (no todos)
- Convertir la fecha a un formato DateTime, de forma que mongo inserte el dato correctamente
- Eliminar campos que no queremos tener en el modelo (id_miembro)

In [8]:
for movimiento_json in json_list:
    movimiento_json['fecha'] = datetime.datetime.fromtimestamp(movimiento_json['fecha'] / 1e3)
    
    del movimiento_json['id_miembro']

    if  movimiento_json['comercio'] == None:
        del movimiento_json['comercio']
    
    db.movimientos.insert_one(movimiento_json)

## Querys

Todas las querys de este caso de uso se pueden resolver directamente por la base de datos ...

### Los 10 movimientos mas caros

In [9]:
rs = db.movimientos.find({}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [10]:
pd.DataFrame(list(rs))

,fecha,importe,actividad_completa,nombre
0,2009-12-30,16921.75,EL CORTE INGLES,Ildefonso José Sánchez Barcoj
1,2006-04-04,12597.26,EL CORTE INGLES,Miguel Blesa de la Parra
2,2003-04-03,5998.35,AGENCIAS DE VIAJES,Ramón Ferraz Ricarte
3,2009-08-23,5909.29,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",Ramón Ferraz Ricarte
4,2010-12-20,4985.00,HIPERCOR SUPERMERCADOS EL CORTE INGLES,Estanislao Rodríguez-Ponga Salamanca
5,2006-08-02,4972.00,AGENCIAS DE VIAJES,Matías Amat Roca
6,2010-10-07,4336.00,AGENCIAS DE VIAJES,María Carmen Cafranga Cavestany
7,2010-03-25,4320.50,EL CORTE INGLES,Ildefonso José Sánchez Barcoj
8,2008-12-19,4218.00,JOYERIAS Y RELOJERIAS,Carmen Contreras Gómez
9,2007-02-11,4175.87,AIR EUROPA,Carmen Contreras Gómez


### Los 10 movimientos mas caros por actividad

In [11]:
rs = db.movimientos.find({
    "actividad": "HOGAR"
}, {
    "nombre": 1,
    "fecha": 1,
    "actividad_completa": 1,
    "importe": 1,
    "_id" : 0
}).limit(10).sort([("importe", -1)])

In [12]:
pd.DataFrame(list(rs))

,fecha,importe,actividad_completa,nombre
0,2009-07-05,3512.00,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",Jesús Pedroche Nieto
1,2011-10-20,3061.00,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Jesús Pedroche Nieto
2,2008-05-15,2764.19,FLORES Y PLANTAS,Francisco Baquero Noriega
3,2006-06-30,1217.94,IKEA,Mariano Pérez Claver
4,2009-01-02,1000.41,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Jesús Pedroche Nieto
5,2011-08-22,945.82,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Jesús Pedroche Nieto
6,2006-09-12,843.25,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Maria Mercedes de la Merced Monge
7,2011-04-02,760.00,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Ildefonso José Sánchez Barcoj
8,2006-07-04,620.00,"MUEBLES,ANTIGUEDADES Y GALERIAS DE ARTE",Alejandro Couceiro Ojeda
9,2007-04-11,575.90,"MATERIALES CONSTRUCCION,FONTANERIA,SANEA",Enrique de la Torre Martínez


### Las 10 personas que mas han gastado

En este caso, al obtener un dato agregado, necesitamos utilizar la función **aggregate()** de mongo

In [13]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$nombre", "importe" : { "$sum" : "$importe"}}},
        {"$sort" : {"importe" : -1}},
        {"$limit" : 10},
        {"$project" : {"_id" : 0, "nombre": "$_id", "importe": 1}}
    ])

In [14]:
df = pd.DataFrame(list(rs))
df

,importe,nombre
0,71331.33,Ildefonso José Sánchez Barcoj
1,60325.29,Miguel Blesa de la Parra
2,46435.74,Ricardo Morado Iglesias
3,45026.71,José Antonio Moral Santín
4,44438.94,Ramón Ferraz Ricarte
5,42663.76,Matías Amat Roca
6,32121.01,Mariano Pérez Claver
7,31558.70,Francisco Baquero Noriega
8,30159.02,Carmen Contreras Gómez
9,27954.38,Antonio Romero Lázaro


### ¿Que comercio es más popular?

Este es un caso especial de agregación, ya que cuenta los distintos. Dentro del flujo de la función se realizan varios pasos para resolver esta query:

- Formar un conjunto con los distintas personas que han comprado en cada comercio
- Desagregar el conjunto
- Agrupar los comercios, contanto los distintos elementos de cada conjunto

In [15]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"},
        {"$group" : { "_id" : "$_id", "cuenta" : { "$sum" : 1}}},
        {"$sort" : {"cuenta" : -1}},
        {"$project" : {"_id" : 0, "comercio": "$_id", "cuenta": 1}},
        {"$match": {"comercio": { "$exists": True, "$ne": None }}},
        {"$limit" : 10}
    ])

In [16]:
df = pd.DataFrame(list(rs))
df

,cuenta,comercio
0,52,EL CORTE INGLES
1,25,IBERIA
2,23,EL CORTE INGLES S.A.
3,22,RENFE 001
4,17,VIAJES ECI
5,15,FNAC CALLAO
6,14,PARKING EGUISA C/SEVILLA
7,13,APARCAMIENTO CORTES
8,12,REST EL ESPIGON
9,10,RESTAURANTE LA ANCHA


Truco ...

En este tipo de querys complicadas podemos ver los resultados parciales ...

In [17]:
rs = db.movimientos.aggregate([
        {"$group" : { "_id" : "$comercio", "nombres" : { "$addToSet" : "$nombre"}}},
        {"$unwind" : "$nombres"}
    ])

pd.DataFrame(list(rs)).head()

,_id,nombres
0,HOTEL BARCELO SANCTRI PET,Carlos María Martínez Martínez
1,BOULANGER ESPANA SL,Domingo Navalmoral Sánchez
2,MADRID CAMISERO,Rafael Spottorno Díaz Caro
3,RESTAURANTE LA ALBUFERA,Ignacio de Navasques Cobián
4,APARCAMIENTO PLAZA DE ESPAA,Antonio Rey de Viñas Sánchez-Majestad
